<a href="https://colab.research.google.com/github/parus-cristatus/tolokapizza/blob/main/tolokaapi/post_bonuses_custom_logic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Docs:** https://toloka.ai/ru/docs/api/concepts/create-bonus.html

In [1]:
import requests
import json

In [2]:
endpoint = "https://sandbox.toloka.yandex.com/api/v1" #Sandbox environment
token = "OAuth AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA"

In [3]:
# configure GET request
def get_call(resource):
    headers = {
        "Authorization": token
    }

    res = requests.get(endpoint + resource, headers=headers)
    return res

In [4]:
# configure POST request
def post_call(resource, data, item: str = None):
    headers = {
        "Authorization": token,
        "Content-Type": "application/JSON"
    }

    res = requests.post(endpoint + resource, headers=headers, data=data)

    return res if item is None else (res, res.json()[item])

In [5]:
# track async operations
def status_operations(id: str):
    print(get_call(f'/operations/{id}/log').text)

In [6]:
# configure POST bonuses request body
def set_post_data(user_id: str, amount: float, message: str, private: str = None, title: str = "Bonuses for Test project.") -> dict:
    return {
        "user_id": user_id, 
        "amount": amount, 
        "private_comment": private, 
        "public_title": {
            "EN": title
            }, 
        "public_message": {
            "EN": message
            }
        }

In [9]:
POOL_ID = 1181996
ASSIGMENT_STATUS = 'SUBMITTED'
operation_ids = []


result = get_call(f'/assignments?status={ASSIGMENT_STATUS}&pool_id={POOL_ID}').json()


def get_unique_ids(assignments_data: dict) -> set:
    """Returns unique user ids
    Arguments:
        assignments_data {dict}: data from get_call function with 'items' key

    Example:
        get_unique_ids(result['items'])
    """
    return {i['user_id'] for i in assignments_data}


def get_num_completed_task(assignments_data: dict, user_id: str) -> int:
    """Returns number of completed tasks
    Arguments:
        assignments_data {dict}: data from get_call function with 'items' key
        user_id {str}: user id

    Example:
        get_num_completed_task(result['items'], '517e4d83f89b2e7b9')
    """
    return [i['user_id'] for i in result['items']].count(user_id)


bonuses = {
    3: {'amount': 0.005, 'message': 'Keep your damn 0.005 cents!'},
    5: {'amount': 0.007, 'message': 'Not bad dude! Here is your 0.007 cents!'},
    9: {'amount': 0.009, 'message': 'Thank you! Now you\'re rich here\'s your 0.009 cents.'}
    }

# Get unique users IDs who submitted assignments in the pool
for id in get_unique_ids(result['items']):
    # if assignments quatity > 3
    if 3 <= get_num_completed_task(result['items'], id) <= 5:
        response = post_call(
            resource='/user-bonuses?async_mode=true&skip_invalid_items=true', 
            data=json.dumps(set_post_data(user_id=id, **bonuses[3])), 
            item='id'
            )
        operation_ids.append(response[1])
    # if assignments quatity > 5
    elif 5 < get_num_completed_task(result['items'], id) <= 8:
        response = post_call(
            resource='/user-bonuses?async_mode=true&skip_invalid_items=true', 
            data=json.dumps(set_post_data(user_id=id, **bonuses[5])), 
            item='id'
            )
        operation_ids.append(response[1])
    # if assignments quatity > 8
    elif 8 < get_num_completed_task(result['items'], id) <= 10:
        response = post_call(
            resource='/user-bonuses?async_mode=true&skip_invalid_items=true', 
            data=json.dumps(set_post_data(user_id=id, **bonuses[9])), 
            item='id'
            )
        operation_ids.append(response[1])

In [ ]:
for id in operation_ids:
   status_operations(id=id)

Get the number of assignments submitted by specific users

In [ ]:
users_id = {i['user_id'] for i in result['items']}
for i in users_id:
    print([i['user_id'] for i in result['items']].count(i))